# fct_operator_hourly_summary

In [1]:
import datetime
import sys

import _operator_grain_route_dir_visuals as _report_operator_visuals
import _operator_grain_scheduled_service 
import _sql_query
import google.auth
import pandas as pd
from google.cloud import bigquery
from IPython.display import HTML, Image, Markdown, display, display_html
from loguru import logger
from omegaconf import OmegaConf
from shared_utils import gtfs_utils_v2, portfolio_utils, publish_utils, rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SEGMENT_GCS

readable_dict = OmegaConf.load("readable2.yml")

import altair as alt
import geopandas as gpd
from shapely import wkt

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_name = "City and County of San Francisco"

In [4]:
df = _sql_query.download_with_pandas_gbq(
    project="cal-itp-data-infra-staging",
    filename="tiffany_mart_gtfs_rollup.fct_operator_hourly_summary",
)


        SELECT 
            *
        FROM `cal-itp-data-infra-staging`.`tiffany_mart_gtfs_rollup`.`fct_operator_hourly_summary`
        WHERE month_first_day >=  DATE('2025-01-01')
    


/opt/conda/lib/python3.11/site-packages/pandas_gbq/gbq.py:38: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa


Downloading: 100%|██████████|
download time: 0:00:02.099819


In [5]:
df.columns

Index(['month_first_day', 'name', 'analysis_name', 'day_type',
       'departure_hour', 'n_trips'],
      dtype='object')

In [6]:
df.sample()

,month_first_day,name,analysis_name,day_type,departure_hour,n_trips
11948,2025-10-01,Amtrak Schedule,Amtrak,Saturday,21,24


## `service_hours` is  missing, there is only `n_trips`. Also departure hour spans above 24?

In [7]:
df2 = (
        df.groupby(["analysis_name", 
                    "month_first_day", 
                    "day_type", 
                    "departure_hour"])
        .agg({"n_trips": "sum"})
        .reset_index()
    )

In [8]:
df2.head()

,analysis_name,month_first_day,day_type,departure_hour,n_trips
0,Alameda-Contra Costa Transit District,2025-08-01,Saturday,0,14
1,Alameda-Contra Costa Transit District,2025-08-01,Saturday,1,14
2,Alameda-Contra Costa Transit District,2025-08-01,Saturday,2,14
3,Alameda-Contra Costa Transit District,2025-08-01,Saturday,3,18
4,Alameda-Contra Costa Transit District,2025-08-01,Saturday,4,26


In [9]:
sf_only = df2.loc[df2.analysis_name == analysis_name]

In [10]:
sf_only.columns = sf_only.columns.str.replace("_", " ").str.title()

In [11]:
saturday = sf_only.loc[sf_only["Day Type"] == "Saturday"]

In [12]:
saturday = saturday.rename(columns = {"Month First Day":"Date"})

In [13]:
saturday["Date"] = saturday.Date.astype(str)

In [14]:
selection = alt.selection_point(fields=["Date"], bind="legend")

In [15]:
date_list = saturday["Date"].unique().tolist()


In [16]:
date_list

['2025-08-01', '2025-09-01', '2025-10-01']

In [17]:
date_dropdown = alt.binding_select(
        options=date_list,
        name="Dates: ",
    )

In [18]:
xcol_param = alt.selection_point(
        fields=["Date"], value=date_list[0], bind=date_dropdown
    )

In [19]:
saturday_chart = (
        alt.Chart(saturday)
        .mark_line(size=3)
        .encode(
            x=alt.X(
                "Departure Hour",
                title="Departure Hour",
                axis=alt.Axis(labelAngle=-45,),
            ),
            y=alt.Y(
                "N Trips",
                title="N Trips",
            ),
        )
    ).add_params(xcol_param).transform_filter(xcol_param)

In [20]:
bg = _operator_grain_scheduled_service.create_bg_service_chart()

In [21]:
chart = (saturday_chart + bg).properties(
        resolve=alt.Resolve(
            scale=alt.LegendResolveMap(color=alt.ResolveMode("independent"))
        )
    )

In [22]:
_report_operator_visuals.configure_chart(
    chart,
    width=400,
    height=250,
    title="Daily Scheduled Service Hours for Saturday",
    subtitle="Description",
)

alt.LayerChart(...)